# Outils de création d'une carte avec la création d'un parcours
Fichier nécessaire: un fichier en format .txt contenant la liste des noms de lieux à recenser sur la carte.

Les outils de recherches d'informations permettent de récupérer les emplacements de chaque lieu (coordonnées GPS), le type de lieu dans lequel il est catégorisé ainsi qu'une brève description du lieu. Un fichier sous la forme d'un tableau lisible avec Excel est créé avec les informations récoltées.

La récupération des coordonnées GPS permet de retracer un parcours.

La carte est enregistrée en format html.

Il est possible à la fin d'ajouter, à la fin du notebook, son propre fond de carte.

**Avant de lancer le notebook**:

Créer un dossier sur son Google Drive intitulé "Creation-de-cartes".
Placer dans ce dossier ce notebook, ainsi que le fichier .txt contenant la liste des lieux.

## Connexion à son compte GoogleDrive:

- Lancer la cellule
- Cliquer sur le lien généré en dessous
Copier le mot de passe dans la nouvelle fenêtre ouverte où l'accès au compte est requis et le coller en dessous de la cellule.

In [2]:
!pip install geopy
!pip install --upgrade wptools
from google.colab import drive
import folium
from geopy.geocoders import Nominatim
import pandas as pd
from folium import plugins
import json
import urllib.parse, urllib.request
import numpy as np
import csv
from collections import Counter
from branca.element import Figure

drive.mount('/content/drive/')
%cd /content/drive/My Drive/NER-demo-lieux/

Requirement already up-to-date: wptools in /usr/local/lib/python3.7/dist-packages (0.4.17)
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/NER-demo-lieux


## Récupération des identifiants WIKIDATA des lieux

- Indiquer le chemin du fichier .txt avec la liste des lieux (fin de la cellule)
+ possibilité de changer le nom du fichier de sortie (fichier .txt avec liste des identifiants)
- Changer la langue si nécessaire (par défaut: français)
- Si la détéction n'est pas assez précis, descendre "thresold" à 0.8 (moins de noms vont être identifiés), dans la cas contraire, monter le seuil à 1. Par défaut, il est placé à une valeur moyenne de 0.9

In [ ]:
def CallWikifier(text, lang="fr", threshold=1): 
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
        print(response)
    # Sortie des annotations:
    with open('link-lieux2.txt', 'w') as outfile:
        for annotation in response["annotations"]:
          ## Récupération (uniquement de l'ID wikidata):
          itemid = annotation["wikiDataItemId"] 
          itemid2 = "".join([str(itemid)," "])
          outfile.write(itemid2)

with open ("test-it-lieux.txt", "r") as myfile:
    liste=myfile.read()
CallWikifier(text=liste, lang="fr")

{'annotations': [{'title': 'Place (voie)', 'url': 'http://fr.wikipedia.org/wiki/Place_(voie)', 'lang': 'fr', 'pageRank': 0.01030158054090963, 'secLang': 'en', 'secTitle': 'Town square', 'secUrl': 'http://en.wikipedia.org/wiki/Town_square', 'wikiDataItemId': 'Q174782', 'wikiDataClasses': [], 'dbPediaTypes': [], 'dbPediaIri': 'http://dbpedia.org/resource/Town_square', 'supportLen': 3, 'support': [{'wFrom': 0, 'wTo': 0, 'chFrom': 0, 'chTo': 4, 'pMentionGivenSurface': 0.01711017953157751, 'pageRank': 0.0004208193773132968, 'prbConfidence': 0.9652415111552961, 'entropy': 0.05889157293723925}, {'wFrom': 10, 'wTo': 10, 'chFrom': 61, 'chTo': 65, 'pMentionGivenSurface': 0.01711017953157751, 'pageRank': 0.0004208193773132968, 'prbConfidence': 0.9652415111552961, 'entropy': 0.05889157293723925}, {'wFrom': 17, 'wTo': 17, 'chFrom': 105, 'chTo': 109, 'pMentionGivenSurface': 0.01711017953157751, 'pageRank': 0.0004208193773132968, 'prbConfidence': 0.9652415111552961, 'entropy': 0.05889157293723925}]},

## Récupération des informations de chaque lieu reconnu et lié à wikidata:

- Lancer la cellule

Toutes les informations (le type de lieu, les coordonnées GPS (latitude et longitude) ainsi qu'une brève description du lieu) sont enregistrées et exportées sous la forme d'un fichier de données JSON ainsi que sous la forme d'un tableau dans un fichier Excel dans le Google Drive. Il est possbile de changer le nom du fichier xlsx créé à la toute dernière ligne de la cellule: ici, il se nomme "informations_lieux.xlsx"

In [3]:
%cd /content/drive/My Drive/NER-demo-lieux/

import wptools
import re
import itertools

donnees ={}
nom = []
type_lieu = []
latitude = []
longitude = []
longitude = []
description = []
# Utilisation du document créé précédement avec les identifiants wikidata:
with open("/content/drive/My Drive/NER-demo-lieux/link-lieux2.txt", "r") as a_file:
      with open("donnees_lieux.json", "w") as fichier : 
        for line in a_file:
            for word in line.split():
              page = wptools.page(wikibase=word, lang="fr")
              page.get_wikidata()
              page.get_parse()
## Pour avoir toutes les informations:
              try:
                infobox=page.data['infobox']
                nom = infobox.get("nom")
                type_lieu=infobox.get("type")
                if type_lieu is None: 
                  try:
                    type_lieu=page.data['what']
                  except KeyError as ke:
                    pass
                latitude = infobox.get("latitude")
                longitude = infobox.get("longitude")
                description = page.data["description"]
              except AttributeError as at:
                print("Ce n'est pas un lieu!")
# Création d'un dictionnaire + fichier json avec les informations récupérées sur les lieux:
              try:
                donnees[nom] = {"Type de lieu" : type_lieu,
                              "Latitude" : latitude, "Longitude" : longitude, "Description": description}
              except TypeError as tpy:
                pass
      
        json.dump(donnees, fichier, ensure_ascii=False)

# Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel:
df = pd.DataFrame(donnees)
df2 = df.transpose()
df2.to_excel('informations_lieux.xlsx')

/content/drive/My Drive/NER-demo-lieux


www.wikidata.org (wikidata) Q174782
www.wikidata.org (labels) P910|P2581|P3827|P9230|P2452|Q98929991|...
Note: Wikidata item Q174782 missing 'instance of' (P31)
fr.wikipedia.org (imageinfo) File:Московский Кремль №5.JPG
Place (voie) (fr) data
{
  claims: <dict(29)> P508, P373, P279, P349, P646, P1051, P1709, P...
  description: espace public non bâti, desservi par des voies
  image: <list(1)> {'file': 'File:Московский Кремль №5.JPG', 'kind...
  label: place
  labels: <dict(38)> P910, P2581, P3827, P9230, P2452, Q98929991, ...
  modified: <dict(1)> wikidata
  requests: <list(3)> wikidata, labels, imageinfo
  title: Place_(voie)
  wikibase: Q174782
  wikidata: <dict(27)> identifiant Bibliothèque nationale centrale...
  wikidata_pageid: 174672
  wikidata_url: https://www.wikidata.org/wiki/Q174782
}
fr.wikipedia.org (parse) Place_(voie)
Place (voie) (fr) data
{
  claims: <dict(29)> P508, P373, P279, P349, P646, P1051, P1709, P...
  description: espace public non bâti, desservi par des voie

Ce n'est pas un lieu!


www.wikidata.org (labels) Q191066|Q175129|Q3252156|P41|P2950|P321...
www.wikidata.org (labels) P3500|Q3260385|P47|Q1032646|Q3262481|Q5...
www.wikidata.org (labels) P9037|P138|P8406|P8687|Q3485975|Q172765...
www.wikidata.org (labels) Q158119|Q209086|P856|Q24575431|P1830|Q1...
www.wikidata.org (labels) P1454|P3106|Q602358|Q4114391|P691|P5247...
www.wikidata.org (labels) Q3485977|P6262|Q601266|Q386991|Q243|Q31...
www.wikidata.org (labels) Q270230|Q208943|Q12761|P1335|Q234743|P1...
www.wikidata.org (labels) Q64|P5785|P6200|Q640102|P982|P8422|P793...
fr.wikipedia.org (imageinfo) File:Paris - Eiffelturm und Marsfeld2.jpg
Paris (fr) data
{
  aliases: <list(5)> Ville-Lumière, Paname, Lutèce, Ville de l'Amo...
  claims: <dict(183)> P2924, P1082, P1667, P1151, P1333, P349, P21...
  description: capitale de la France
  image: <list(1)> {'file': 'File:Paris - Eiffelturm und Marsfeld2...
  label: Paris
  labels: <dict(381)> Q191066, Q175129, Q3252156, P41, P2950, P321...
  modified: <dict(1)> wikid

Ce n'est pas un lieu!


...

www.wikidata.org (labels) P910|P4672|P1566|P41|Q193899|P6671|P785...
www.wikidata.org (labels) Q668437|P94|P268|P1889|P5573|P4212|P856...
fr.wikipedia.org (imageinfo) File:Château de Vincennes Paris FRA ...
Vincennes (fr) data
{
  claims: <dict(59)> P1036, P190, P910, P373, P374, P17, P94, P625...
  description: commune française du département du Val-de-Marne
  image: <list(1)> {'file': 'File:Château de Vincennes Paris FRA 0...
  label: Vincennes
  labels: <dict(86)> P910, P4672, P1566, P41, Q193899, P6671, P785...
  modified: <dict(1)> wikidata
  requests: <list(4)> wikidata, labels, labels, imageinfo
  title: Vincennes
  what: commune de France
  wikibase: Q193819
  wikidata: <dict(57)> classification décimale de Dewey (P1036), j...
  wikidata_pageid: 191794
  wikidata_url: https://www.wikidata.org/wiki/Q193819
}
fr.wikipedia.org (parse) Vincennes
fr.wikipedia.org (imageinfo) File:Donjon du Château de Vincennes.jpg
Vincennes (fr) data
{
  claims: <dict(59)> P1036, P190, P910, P373, 

Ce n'est pas un lieu!


www.wikidata.org (labels) P910|P2581|P3827|P9230|P2452|P646|P5357...
www.wikidata.org (labels) Q2657718|P1225|P1014|P1417|Q1401536|P12...
Note: Wikidata item Q43501 missing 'instance of' (P31)
fr.wikipedia.org (imageinfo) File:Przewalskis horses exposition, ...
Parc zoologique (fr) data
{
  aliases: <list(3)> zoo, jardin zoologique, Zoo jardin
  claims: <dict(57)> P1282, P373, P508, P279, P910, P349, P227, P6...
  description: espace où sont réunies de nombreuses espèces animales
  image: <list(1)> {'file': 'File:Przewalskis horses exposition, Z...
  label: parc zoologique
  labels: <dict(68)> P910, P2581, P3827, P9230, P2452, P646, P5357...
  modified: <dict(1)> wikidata
  requests: <list(4)> wikidata, labels, labels, imageinfo
  title: Parc_zoologique
  wikibase: Q43501
  wikidata: <dict(56)> attribut ou clé OpenStreetMap (P1282), caté...
  wikidata_pageid: 45694
  wikidata_url: https://www.wikidata.org/wiki/Q43501
}
fr.wikipedia.org (parse) Parc_zoologique


Ce n'est pas un lieu!


Parc zoologique (fr) data
{
  aliases: <list(3)> zoo, jardin zoologique, Zoo jardin
  claims: <dict(57)> P1282, P373, P508, P279, P910, P349, P227, P6...
  description: espace où sont réunies de nombreuses espèces animales
  image: <list(1)> {'file': 'File:Przewalskis horses exposition, Z...
  iwlinks: <list(12)> https://en.wikipedia.org/wiki/Greek_gardens,...
  label: parc zoologique
  labels: <dict(68)> P910, P2581, P3827, P9230, P2452, P646, P5357...
  modified: <dict(1)> wikidata
  pageid: 114061
  parsetree: <str(55336)> <root><template><title>confusion</title>...
  requests: <list(5)> wikidata, labels, labels, imageinfo, parse
  title: Parc zoologique
  wikibase: Q43501
  wikidata: <dict(56)> attribut ou clé OpenStreetMap (P1282), caté...
  wikidata_pageid: 45694
  wikidata_url: https://www.wikidata.org/wiki/Q43501
  wikitext: <str(45315)> {{confusion|Parc animalier}}{{Voir homony...
}


##Outils de calculs statistiques:

Calcul des types de lieux détectés sur wikidata ainsi que la proportion de chaque.

- Lancer la cellule.

- Les résultats s'affichent en dessous : le nombre de lieux par "type" ainsi que la proportion de chaque catégorie.

In [ ]:
# Récupération et création d'une liste avec catégories de lieux
keys = donnees[nom]["Type de lieu"]
liste_des_types = []
fichier_json = "/content/drive/My Drive/NER-demo-lieux/donnees_lieux.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)


def proportion_types(liste):
  c = Counter(liste)
  print("Voici le nombre de lieux, triés par chaque type:", c) ## Affiche le nombre de lieux de chaque type
  prop = [(i, c[i] / len(liste) * 100.0) for i in c]
  print("Voici la proportion de chaque type de lieux:", prop) ## Affiche la proportion de chaque type de lieux
  
proportion_types(liste_des_types)



Voici le nombre de lieux, triés par chaque type: Counter({'capitale': 1, None: 1, 'taxon': 1, 'station de métro': 1, 'Château fort, arsenal et prison': 1, '[[Boisson énergisante]] [[psychotrope]] [[stimulant]]e': 1, 'commune de France': 1})
Voici la proportion de chaque type de lieux: [('capitale', 14.285714285714285), (None, 14.285714285714285), ('taxon', 14.285714285714285), ('station de métro', 14.285714285714285), ('Château fort, arsenal et prison', 14.285714285714285), ('[[Boisson énergisante]] [[psychotrope]] [[stimulant]]e', 14.285714285714285), ('commune de France', 14.285714285714285)]


##Création d'une carte avec un parcours

In [6]:

keys_lat = donnees[nom]["Latitude"]
keys_long = donnees[nom]["Longitude"]
liste_des_lat = []
liste_des_long = []
fichier_json = "/content/drive/My Drive/NER-demo-lieux/donnees_lieux.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_lat=value["Latitude"]
      liste_des_lat.append(test_lat)
      test_long=value["Longitude"]
      liste_des_long.append(test_long)

coords = []
for ju, ij in zip(liste_des_lat, liste_des_long):
  if ju is None:
    pass  
  elif ij is None:
    pass
  else:
    coords.append([float(ju), float(ij)])
print(coords) ### Crée un array de coordonnées en suppirimant les None

fig5=Figure(height=550,width=750)
m5=folium.Map(location=[48.856614, 2.3522219], zoom_start=12)
fig5.add_child(m5)

f1=folium.FeatureGroup("Parcours")
line_1=folium.vector_layers.PolyLine(coords,popup='<b>Parcours_test</b>',tooltip='Tour de France',color='blue',weight=4).add_to(f1)

f1.add_to(m5)
folium.LayerControl().add_to(m5)
m5

1000


## Créer et ajouter son fond de carte

Si l'on veut ajouter son PROPRE fond de carte personnalisé:

- Création d’un compte sur : https://mapwarper.net/

- Téléchargement du fond de carte (IIIF recommandé, avec la meilleure résolution possible)

- Sur Map Warper: dans l’onglet « Upload Map » il est possible de charger le fond de carte que l’on veut utiliser, et indiquer toutes les métadonnées nécessaires.

- Dans l’onglet "Rectify": il faut indiquer le plus de points identiques sur les deux cartes pour permettre la superposition du fond de carte chargé sur des données géolocalisées. Cliquer sur un point sur chaque carte, puis cliquer sur « add Control point » en dessous des cartes. Répéter l’opération le plus de fois possible pour obtenir un bon résultat. Dans l'encadré en dessous des deux fonds de carte, cliquer sur "Warp image": le fond s'ajoute. Si le résultat n'est pas satisfaisant, il possbile d'ajouter d'autres repères pour améliorer la qualité du nouveau fond de carte créé. 

- Se rendre dans l'onglet "Export". Dans "Map services", copier le lien entier se situant après "Tiles (Google/OSM scheme):", débutant par http et se terminant par .png

- Coller ensuite le lien à la troisième ligne de la cellule suivante, à la place du lien existant, après "tiles = " en conservant bien les guillemets.

In [7]:
fig5=Figure(height=550,width=750)
m5=folium.Map(location=[48.856614, 2.3522219], zoom_start=12,
                 tiles="https://mapwarper.net/maps/tile/54525/{z}/{x}/{y}.png", attr="None")

keys_lat = donnees[nom]["Latitude"]
keys_long = donnees[nom]["Longitude"]
liste_des_lat = []
liste_des_long = []
fichier_json = "/content/drive/My Drive/NER-demo-lieux/donnees_lieux.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_lat=value["Latitude"]
      liste_des_lat.append(test_lat)
      test_long=value["Longitude"]
      liste_des_long.append(test_long)

coords = []
for ju, ij in zip(liste_des_lat, liste_des_long):
  if ju is None:
    pass  
  elif ij is None:
    pass
  else:
    coords.append([float(ju), float(ij)])
print(coords) ### Crée un array de coordonnées en suppirimant les None

fig5.add_child(m5)

f1=folium.FeatureGroup("Parcours")
line_1=folium.vector_layers.PolyLine(coords,popup='<b>Parcours_test</b>',tooltip='Tour de France',color='blue',weight=4).add_to(f1)

f1.add_to(m5)
folium.LayerControl().add_to(m5)
m5

[[48.856578, 2.351828], [48.831483, 2.355692], [48.853333, 2.369167], [48.8477777778, 2.43916666667]]
